In [1]:
import numpy as np
import pandas as pd
from collections import defaultdict

import pymongo
from pymongo import MongoClient

import time
import requests
import os
import re
from bs4 import BeautifulSoup
import json

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
global chrome_options
PROXY = ''
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--proxy-server=%s' % PROXY)

In [3]:
import time
from selenium import webdriver

driver = webdriver.Chrome('/Users/CEO/Downloads/chromedriver_2')  # Optional argument, if not specified will search path.
driver.get('http://www.google.com/xhtml');

In [4]:
time.sleep(5) # Let the user actually see something!

In [5]:
search_box = driver.find_element_by_name('q')
search_box.send_keys('ChromeDriver')
search_box.submit()

In [6]:
time.sleep(5) # Let the user actually see something!
driver.quit()

In [7]:
webpage = requests.get('https://www.whoscored.com/Regions/252/Tournaments/2/Seasons/1849/Stages/3115/Fixtures/England-Premier-League-2009-2010')

In [8]:
soup = BeautifulSoup(webpage.text, 'html.parser')
soup

<html style="height:100%"><head><meta content="NOINDEX, NOFOLLOW" name="ROBOTS"/><meta content="telephone=no" name="format-detection"/><meta content="initial-scale=1.0" name="viewport"/><meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/></head><body style="margin:0px;height:100%"><iframe frameborder="0" height="100%" marginheight="0px" marginwidth="0px" src="/_Incapsula_Resource?CWUDNSAI=20&amp;xinfo=8-6925502-0%200NNN%20RT%281542737553805%2047%29%20q%280%20-1%20-1%20-1%29%20r%280%20-1%29%20B15%284%2c200%2c0%29%20U2&amp;incident_id=515001370037581187-46695301087692952&amp;edet=15&amp;cinfo=04000000" width="100%">Request unsuccessful. Incapsula incident ID: 515001370037581187-46695301087692952</iframe></body></html>

In [9]:
def get_result2(data_dict, html_list):
    head = 'https://www.whoscored.com'
    reports = []
    for soup in html_list:
        tags_home = soup.findAll('td', class_="team home winner")
        tags_result = soup.findAll('a', class_="result-1 rc")
        tags_away = soup.findAll('td', class_="team away")
        tags_match_report = soup.findAll('a', class_="match-link match-report rc")
        
        for tag_home,tag_result, tag_away, tag_match_report in zip(tags_home, tags_result, tags_away, tags_match_report):
            data_dict['home_teams'].append(tag_home.contents[0].text)
            
            result = tag_result.text
            array = result.split(':')
            data_dict['home_results'].append(int(array[0]) - int(array[1]))
            
            data_dict['away_teams'].append(tag_away.contents[0].text)
            data_dict['match_reports'].append(tag_match_report['href'])
            data_dict['id'].append(int(tag_match_report['href'].split('/')[2]))
            
            #reports.append((head + tag_match_report['href']).replace('Show', 'MatchReport'))
            reports.append((head + tag_match_report['href']))
    return data_dict, reports

In [10]:
def get_html(season_url, months):
    driver_s = webdriver.Chrome('/Users/CEO/Downloads/chromedriver_2')
    html_list = []
    
    while True:
        try:
            driver_s.get(season_url)
            break
        except:
            driver_s.close()
            driver_s = webdriver.Chrome('/Users/CEO/Downloads/chromedriver_2')

    for month in months:
        time.sleep(5)
        fixture_html = driver_s.page_source
        fixture_soup = BeautifulSoup(fixture_html, 'html.parser')
        html_list.append(fixture_soup)
        if month != months[-1]:
            while True:
                try:
                    driver_s.find_element_by_css_selector('#date-controller > a.previous.button.ui-state-default.rc-l.is-default').click()
                    break
                except:
                    driver_s.refresh()
                    time.sleep(10)

    driver_s.close()
    return html_list

def get_result(data_dict, html_list):
    head = 'https://www.whoscored.com'
    reports = []
    for soup in html_list:
        tags_home = soup.select('#tournament-fixture > tbody > tr > td.team.home')
        tags_result = soup.select('#tournament-fixture > tbody > tr > td.result')
        tags_away = soup.select('#tournament-fixture > tbody > tr > td.team.away')
        tags_match_report = soup.select('#tournament-fixture > tbody > tr > td.toolbar.right > a.match-link.match-report.rc')
        
        for tag_home,tag_result, tag_away, tag_match_report in zip(tags_home, tags_result, tags_away, tags_match_report):
            data_dict['home_teams'].append(tag_home.contents[-1].text)
            
            result = tag_result.text
            array = result.split(':')
            data_dict['results'].append(int(array[0]) - int(array[1]))
            
            data_dict['away_teams'].append(tag_away.contents[0].text)
            data_dict['id'].append(int(tag_match_report['href'].split('/')[2]))
            
            reports.append((head + tag_match_report['href']).replace('Show', 'MatchReport'))
    return data_dict, reports

def get_general_stats(data_dict, reports_):
    #driver_s = webdriver.Firefox()
    driver_s = webdriver.Chrome('/Users/CEO/Downloads/chromedriver_2')
    reports_iter = list(reports_)
    for report in reports_iter:
        while True:
            try:
                driver_s.get(report)
                time.sleep(4)
                html = driver_s.page_source
                soup = BeautifulSoup(html, 'html.parser')
                data_dict['home_shots'].append(int(soup.select('#match-report-team-statistics > div.stat-group.no-top-margin > div > span')[0].text))
                break
            except:
                driver_s.close()
                driver_s = webdriver.Chrome('/Users/CEO/Downloads/chromedriver_2')
                #driver_s = webdriver.Firefox()
                
        data_dict['away_shots'].append(int(soup.select('#match-report-team-statistics > div.stat-group.no-top-margin > div > span')[2].text))
        data_dict['home_SOT'].append(int(soup.select('#match-report-team-statistics > div.stat-group.no-top-margin > div > span')[3].text))
        data_dict['away_SOT'].append(int(soup.select('#match-report-team-statistics > div.stat-group.no-top-margin > div > span')[5].text))
        data_dict['home_pass_success'].append(int((soup.select('#match-report-team-statistics > div.stat-group.no-top-margin > div > span')[6].text)[:-1])/100.0)
        data_dict['away_pass_success'].append(int((soup.select('#match-report-team-statistics > div.stat-group.no-top-margin > div > span')[8].text)[:-1])/100.0)
        data_dict['home_aerial_success'].append(int((soup.select('#match-report-team-statistics > div.stat-group.no-top-margin > div > span')[9].text)[:-1])/100.0)
        data_dict['away_aerial_success'].append(int((soup.select('#match-report-team-statistics > div.stat-group.no-top-margin > div > span')[11].text)[:-1])/100.0)
        data_dict['home_dribbles'].append(int(soup.select('#match-report-team-statistics > div.stat-group.no-top-margin > div > span')[12].text))
        data_dict['away_dribbles'].append(int(soup.select('#match-report-team-statistics > div.stat-group.no-top-margin > div > span')[14].text))
        data_dict['home_possession'].append(int(soup.select('#match-report-team-statistics > div > div.stat > span > span > span')[1].text[:-1])/100.)
        data_dict['away_possession'].append(int(soup.select('#match-report-team-statistics > div > div.stat > span > span > span')[2].text[:-1])/100.)
        data_dict['home_counter'].append(int(soup.select('#live-goals-content-comparision > div > div > span > span')[6].text)/float(soup.select('#live-goals-content-comparision > div > div > span > span')[0].text))
        data_dict['away_counter'].append(int(soup.select('#live-goals-content-comparision > div > div > span > span')[7].text)/float(soup.select('#live-goals-content-comparision > div > div > span > span')[1].text))
        data_dict['home_setpiece'].append(int(soup.select('#live-goals-content-comparision > div > div > span > span')[4].text)/float(soup.select('#live-goals-content-comparision > div > div > span > span')[0].text))
        data_dict['away_setpiece'].append(int(soup.select('#live-goals-content-comparision > div > div > span > span')[5].text)/float(soup.select('#live-goals-content-comparision > div > div > span > span')[1].text))
        time.sleep(5)
        
        while True:
            try:
                driver_s.find_element_by_css_selector('#live-chart-stats-options > li:nth-child(2) > a').click()
                time.sleep(5)
                html = driver_s.page_source
                soup = BeautifulSoup(html, 'html.parser')
                data_dict['home_total_pass'].append(int(soup.select('#live-passes-content-comparision > div > div > span > span')[0].text))
                break
            except:
                driver_s.refresh()
                time.sleep(10)
                    
        data_dict['away_total_pass'].append(int(soup.select('#live-passes-content-comparision > div > div > span > span')[1].text))
        data_dict['home_crosses'].append(int(soup.select('#live-passes-content-comparision > div > div > span > span')[2].text)/float(data_dict['home_total_pass'][-1]))
        data_dict['away_crosses'].append(int(soup.select('#live-passes-content-comparision > div > div > span > span')[3].text)/float(data_dict['away_total_pass'][-1]))
        data_dict['home_through_pass'].append(int(soup.select('#live-passes-content-comparision > div > div > span > span')[4].text)/float(data_dict['home_total_pass'][-1]))
        data_dict['away_through_pass'].append(int(soup.select('#live-passes-content-comparision > div > div > span > span')[5].text)/float(data_dict['away_total_pass'][-1]))
        data_dict['home_long_pass'].append(int(soup.select('#live-passes-content-comparision > div > div > span > span')[6].text)/float(data_dict['home_total_pass'][-1]))
        data_dict['away_long_pass'].append(int(soup.select('#live-passes-content-comparision > div > div > span > span')[7].text)/float(data_dict['away_total_pass'][-1]))
        data_dict['home_short_pass'].append(int(soup.select('#live-passes-content-comparision > div > div > span > span')[8].text)/float(data_dict['home_total_pass'][-1]))
        data_dict['away_short_pass'].append(int(soup.select('#live-passes-content-comparision > div > div > span > span')[9].text)/float(data_dict['away_total_pass'][-1]))
        data_dict['home_pass_streak'].append(int(soup.select('#live-passes-info > div > div > span > span > span')[5].text))
        data_dict['away_pass_streak'].append(int(soup.select('#live-passes-info > div > div > span > span > span')[6].text))

        while True:
            try:
                driver_s.find_element_by_css_selector('#sub-navigation > ul > li:nth-child(4) > a').click()
                time.sleep(8)
                html = driver_s.page_source
                soup = BeautifulSoup(html, 'html.parser')
                data_dict['home_formation'].append(soup.select('#match-centre-header > div > div.team-info > div.formation')[0].text)
                break
            except:
                driver_s.refresh()
                time.sleep(10)

        data_dict['away_formation'].append(soup.select('#match-centre-header > div > div.team-info > div.formation')[1].text)
        data_dict['home_success_tackles'].append(int(soup.select('#match-centre-stats > ul > li > div.match-centre-stats > ul > li > div > span.match-centre-stat-value')[38].text))
        data_dict['away_success_tackles'].append(int(soup.select('#match-centre-stats > ul > li > div.match-centre-stats > ul > li > div > span.match-centre-stat-value')[39].text))
        data_dict['home_clearances'].append(int(soup.select('#match-centre-stats > ul > li > div.match-centre-stats > ul > li > div > span.match-centre-stat-value')[46].text))
        data_dict['away_clearances'].append(int(soup.select('#match-centre-stats > ul > li > div.match-centre-stats > ul > li > div > span.match-centre-stat-value')[47].text))
        data_dict['home_interceptions'].append(int(soup.select('#match-centre-stats > ul > li > div.match-centre-stats > ul > li > div > span.match-centre-stat-value')[48].text))
        data_dict['away_interceptions'].append(int(soup.select('#match-centre-stats > ul > li > div.match-centre-stats > ul > li > div > span.match-centre-stat-value')[49].text))
        
        while True:
            try:
                driver_s.find_element_by_css_selector('#live-match-options > li:nth-child(3) > a > span:nth-child(2)').click()
                time.sleep(8)
                html = driver_s.page_source
                soup = BeautifulSoup(html, 'html.parser')
                home_6yd = int(soup.select('#chalkboard > div.filterz-filter-groups.clearfix > div.filterz-filter-group.selected > div > div > span.filterz-value')[12].text)
                break
            except:
                driver_s.refresh()
                time.sleep(10)
                   
        away_6yd = int(soup.select('#chalkboard > div.filterz-filter-groups.clearfix > div.filterz-filter-group.selected > div > div > span.filterz-value')[13].text)
        home_pen = int(soup.select('#chalkboard > div.filterz-filter-groups.clearfix > div.filterz-filter-group.selected > div > div > span.filterz-value')[14].text)
        away_pen = int(soup.select('#chalkboard > div.filterz-filter-groups.clearfix > div.filterz-filter-group.selected > div > div > span.filterz-value')[15].text)
        home_out = int(soup.select('#chalkboard > div.filterz-filter-groups.clearfix > div.filterz-filter-group.selected > div > div > span.filterz-value')[16].text)
        away_out = int(soup.select('#chalkboard > div.filterz-filter-groups.clearfix > div.filterz-filter-group.selected > div > div > span.filterz-value')[17].text)
        total_shots = float(home_6yd + away_6yd + home_pen + away_pen + home_out + away_out)
        data_dict['home_6yd_box'].append(home_6yd/total_shots)
        data_dict['away_6yd_box'].append(away_6yd/total_shots)
        data_dict['home_pen_box'].append(home_pen/total_shots)
        data_dict['away_pen_box'].append(away_pen/total_shots)
        data_dict['home_out_box'].append(home_out/total_shots)
        data_dict['away_out_box'].append(home_out/total_shots)
        reports_.pop(0)
    driver_s.close()
    return data_dict

In [11]:
url_09_10 = 'https://www.whoscored.com/Regions/252/Tournaments/2/Seasons/1849/Stages/3115/Fixtures/England-Premier-League-2009-2010'
url_10_11 = 'https://www.whoscored.com/Regions/252/Tournaments/2/Seasons/2458/Stages/4345/Fixtures/England-Premier-League-2010-2011'
url_11_12 = 'https://www.whoscored.com/Regions/252/Tournaments/2/Seasons/2935/Stages/5476/Fixtures/England-Premier-League-2011-2012'
url_12_13 = 'https://www.whoscored.com/Regions/252/Tournaments/2/Seasons/3389/Stages/6531/Fixtures/England-Premier-League-2012-2013'
url_13_14 = 'https://www.whoscored.com/Regions/252/Tournaments/2/Seasons/3853/Stages/7794/Fixtures/England-Premier-League-2013-2014'
url_14_15 = 'https://www.whoscored.com/Regions/252/Tournaments/2/Seasons/4311/Stages/9155/Fixtures/England-Premier-League-2014-2015'
url_15_16 = 'https://www.whoscored.com/Regions/252/Tournaments/2/Seasons/5826/Stages/12496/Fixtures/England-Premier-League-2015-2016'
url_16_17 = 'https://www.whoscored.com/Regions/252/Tournaments/2/Seasons/6335/Stages/13796/Fixtures/England-Premier-League-2016-2017'
urls = [url_09_10, url_10_11, url_11_12, url_12_13, url_13_14, url_14_15, url_15_16, url_16_17]
months = ['May', 'April', 'March', 'Feb', 'Jan', 'Dec', 'Nov', 'Oct', 'Sep', 'Aug']
df_EPL = pd.DataFrame()

In [12]:
html_list = get_html(url_16_17, months)

In [ ]:
len(html_list)

In [ ]:
html_list[1]

In [ ]:
data_dict = defaultdict(list)

In [ ]:
data_dict

In [ ]:
data_dict, reports = get_result(data_dict, html_list)
print ('Results and reports complete!')
print (len(reports))

In [ ]:
len(data_dict['home_teams']), len(data_dict['away_teams']), \
len(data_dict['results']), len(data_dict['id']), len(reports)

In [ ]:
reports

In [ ]:
data_dict2 = defaultdict(list)
data_dict2, reports2 = get_result2(data_dict2, html_list)
print ('Results and reports complete!')
print (len(reports2))

In [ ]:
len(data_dict2['home_teams']), len(data_dict2['away_teams']), \
len(data_dict2['home_results']), len(data_dict2['match_reports']), len(data_dict2['id']), len(reports2)

In [ ]:
reports_ = list(reports)

In [ ]:
data_dict = get_general_stats(data_dict, reports_)
print ('General stats complete!')
time.sleep(5)

In [ ]:
head = 'https://www.whoscored.com'
reports = []
for soup in html_list:
    tags_home = soup.select('#tournament-fixture > tbody > tr > td.team.home')
    tags_result = soup.select('#tournament-fixture > tbody > tr > td.result')
    tags_away = soup.select('#tournament-fixture > tbody > tr > td.team.away')
    tags_match_report = soup.select('#tournament-fixture > tbody > tr > td.toolbar.right > a.match-link.match-report.rc')

print(soup.find_all('td', class_="team home winner")[0])
print()
print(tags_home[0])
print()
print(tags_home[0].contents)
print()
print(tags_home[0].contents[0])
print()
print(tags_home[0].contents[-1])
print()
print(tags_home[0].contents[0].text)
print()
print(tags_home[0].contents[-1].text)
print()
print()
print(tags_result[0])  
print()
print(soup.find_all('a', class_="result-1 rc")[0])
print()
print(tags_result[0].text)  
print()
print(int(tags_result[0].text.split(':')[0]) - 
      int(tags_result[0].text.split(':')[1]))  
print()
print(soup.find_all('td', class_="team away")[0])
print()
print(tags_away[0])  
print()
print(tags_away[0].contents[0].text)
print()
print(soup.findAll('a', class_="match-link match-report rc")[0])  
print()
print(tags_match_report[0])  
print()
print(tags_match_report[0]['href'])  
print()
print(int(tags_match_report[0]['href'].split('/')[2]))
print()
print((tags_match_report[0]['href']).replace('MatchReport', 'Show'))
print()
print((tags_match_report[0]['href']).replace('Show', 'MatchReport'))

## Feature Descriptions